# Libraries

In [1]:
# %cd /users/keimy/git/JOIR_age

# !pip install -r requirements_20221118.txt
# !pip install pandas

# # pytorchをダウンロードし直す
# !pip uninstall torch torchvision torchaudio -y
# !pip install --pre torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/nightly/cpu

In [3]:
import numpy as np
import pandas as pd

import torch
import torchvision.transforms as transforms
import torch.utils.data as data
import timm

from PIL import Image
from tqdm.notebook import tqdm

# 乱数固定化の定義
def torch_seed(seed=1):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.use_deterministic_algorithms = True


# Loading model

In [4]:
torch.backends.mps.is_available()

True

In [5]:
# モデル枠組み読み込み
model = timm.create_model(model_name='swin_base_patch4_window12_384', num_classes=1, pretrained=False)

# GPU使用する場合
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

model = model.to(device)

# 学習済みモデル読み込み
model_path = 'model_20220903.pth'

# GPU使用する場合
# model.load_state_dict(torch.load(model_path))
# CPUで妥協
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

/opt/miniconda3/envs/joir/lib/python3.8/site-packages/torch/functional.py:513: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3610.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/var/folders/lm/x817mz9j3dd69gvsjyjmyhcr0000gn/T/ipykernel_22858/4057488356.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be lo

<All keys matched successfully>

# Inference (sample images)

In [6]:
import glob

# img_list = ['test1.jpg', 'test2.jpg']
img_list = glob.glob('datasets/figures/*.jpg')

# imageNetに合わせた画像の正規化
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

# transformの定義
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean,std),
])

# Datasetクラスの作成
class Dataset(data.Dataset):
    def __init__(self, img_list, transform=None):
        self.img_list = img_list
        self.transform = transform
    
    def __len__(self):
        return len(self.img_list)
    
    def __getitem__(self, index):
        img_path = self.img_list[index]
        img = Image.open(img_path)
        img = self.transform(img)   
        return img

# Datasetの作成
dataset = Dataset(
    img_list=img_list, transform=transform
)

# Dataloaderの作成
# batch sizeはGPU(CPU)性能に応じて適宜変更して下さい
loader = data.DataLoader(
    dataset, batch_size=1, shuffle=False
)

# 乱数固定化
torch_seed()

# 年齢予測
pred_r = []

model.eval()
with torch.no_grad():
    for inputs in tqdm(loader):
        inputs = inputs.to(device)
        outputs = model(inputs)
        pred_r.append(outputs.data.cpu().numpy())
        
pred = np.concatenate(pred_r)

# 結果出力
print(pred)

  0%|          | 0/2 [00:00<?, ?it/s]

[[49.743153]
 [39.084404]]


# Inference (private images)

1000枚で2分くらい。CPUで。

In [11]:
import torchvision.transforms as transforms
from PIL import Image
from tqdm import tqdm
import pandas as pd


# 画像データに対する正規化
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

# transformの定義（リサイズ + 正規化）
transform = transforms.Compose([
    transforms.ToPILImage(),  # numpy -> PIL.Image
    transforms.Resize((384, 384)),  # 画像サイズをモデルに合わせてリサイズ
    transforms.ToTensor(),  # PIL.Image -> Tensor
    transforms.Normalize(mean, std),  # 正規化
])

data = np.load("datasets/mini_fundus_images.npy" , allow_pickle=True)
id_list = data["ID"]
laterality_list = data["laterality"]
image_list = data["image"]


pred_list = []
model.eval()
for img in tqdm(image_list):
    img = transform(img)
    # バッチ次元を追加し、デバイスに転送
    img = img.unsqueeze(0).to(device)
    output = model(img)
    pred_list.append(float(output.item()))


df = pd.DataFrame({
    "ID": id_list,
    "laterality": laterality_list,
    "pred_age": pred_list
})

df.to_csv("outputs/pred_age.csv", index=False)
df.head()

100%|██████████| 1000/1000 [02:21<00:00,  7.09it/s]


leagcy

In [ ]:
import torch
import numpy as np
from torchvision import transforms
from PIL import Image
from tqdm import tqdm

# 画像データに対する正規化
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

# transformの定義（リサイズ + 正規化）
transform = transforms.Compose([
    transforms.ToPILImage(),  # numpy -> PIL.Image
    transforms.Resize((384, 384)),  # モデルに合わせてリサイズ
    transforms.ToTensor(),  # PIL.Image -> Tensor
    transforms.Normalize(mean, std),  # 正規化
])

# データ読み込み
data = np.load("datasets/mini_fundus_images.npy", allow_pickle=True)
image_list = data["image"]

# 画像をすべて変換
transformed_images = torch.stack([transform(img) for img in tqdm(image_list)])  # shape: (N, C, H, W)

# バッチ処理
batch_size = 32  # 適切なバッチサイズを設定
loader = torch.utils.data.DataLoader(transformed_images, batch_size=batch_size, shuffle=False)

# 推論
pred_list = []
model.eval()
with torch.no_grad():
    for batch in tqdm(loader):
        batch = batch.to(device)
        outputs = model(batch)
        pred_list.extend(outputs.squeeze().cpu().tolist())  # バッチ内の結果をリストに追加

print(pred_list)
